In [1]:
from dotenv import load_dotenv

_ = load_dotenv()
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [18]:
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "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"

In [2]:
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file
openai_api_key = os.getenv("OPENAI_API_KEY")


from langchain.tools import tool
import uuid



from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

- Prompts Definition: Define two different prompt templates for replacing an existing address and adding a new address.
- LLM Instance: Create an instance of the language model (ChatOpenAI).
- Memory Buffer: Use ConversationBufferMemory to handle the chat history.
- LLM Chains: Create LLMChain objects for both prompts, integrating the memory buffer.
- Runnable Objects: Create RunnableLambda objects to wrap the chain runs.
- Parallel Execution: Use RunnableParallel to run both chains in parallel.
- Function Binding: Bind functions for address extraction.
- Agent Executor: Integrate everything into AgentExecutor, using the memory buffer to maintain context.

In [3]:
#depreciated
# Define the combined chat prompt template for replacing an existing address
replace_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with replacing address with an exisiting one. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of replacing address with an exisiting one.\n"
        "5. If the user asks about the conversation so far answer within the context.\n"
        "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
        "\n"
        "If the user wants to replace an existing address, follow these steps:\n"
        "Step 1: If the user wants to replace an existing address, ask for the contract ID.\n"
        "Step 2: Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "Step 3: Ask the user to select one address to replace.\n"
        "Step 4: Confirm the choice and generate a reference ID.\n"
        "Step 5: Respond with: 'Great, we have received your request to replace with an existing address. Kindly note your reference ID [reference_id] for further communications.'\n"
        "\n"
        "Example Conversation:\n"
        "User: "
        "Assistant: Would you like to replace address with an existing address?\n"
        "User: Replace address\n"
        "Assistant: Provide me your contract ID.\n"
        "User: ABC1003\n"
        "Assistant: Fetches the list of addresses with the contract ID from a database and displays:\n"
        "1. 123 Main St, Springfield, IL, 62704\n"
        "2. 456 Oak St, Springfield, IL, 62705\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace with an existing address. Kindly note your reference ID abc12234 for further communications.\n"
    ),
    ("user", "{user_input}"),
])

In [4]:
replace_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with replacing an address from an existing list of addresses. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of replacing an address from an existing list of addresses.\n"
        "5. If the user asks about the conversation so far, answer within the context.\n"
        "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
        "7. If the user expresses confusion or asks for repetition, politely clarify and repeat the necessary information.\n"
        "8. If the user wants to go back one step, provide the previous stage's information.\n"
        "9. If the user selects an invalid option, politely ask them to select a valid option from the list.\n"
        "\n"
        "Example Conversations:\n"
        "\n"
        "Example 1:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: DEF4567\n"
        "Assistant: Fetching the list of addresses associated with contract ID DEF4567...\n"
        "1. 789 Pine St, Springfield, IL, 62706\n"
        "2. 101 Maple St, Springfield, IL, 62707\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID def56789 for further communications.\n"
        "\n"
        "Example 2:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: GHI8901\n"
        "Assistant: Fetching the list of addresses associated with contract ID GHI8901...\n"
        "1. 234 Elm St, Springfield, IL, 62708\n"
        "2. 345 Cedar St, Springfield, IL, 62709\n"
        "User: What time does the office close?\n"
        "Assistant: Please stay within the context of the conversation. Would you like to replace an address? Please select one address from the list.\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID ghi89101 for further communications.\n"
        "\n"
        "Example 3:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: JKL1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID JKL1234...\n"
        "1. 456 Birch St, Springfield, IL, 62710\n"
        "2. 567 Walnut St, Springfield, IL, 62711\n"
        "User: What did we talk about earlier?\n"
        "Assistant: We were discussing replacing your address. You provided contract ID JKL1234. Please select one address from the list.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID jkl12345 for further communications.\n"
        "\n"
        "Example 4:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: MNO5678\n"
        "Assistant: The provided contract ID does not match any records. Please check the contract ID and provide it again.\n"
        "User: MNO8765\n"
        "Assistant: Fetching the list of addresses associated with contract ID MNO8765...\n"
        "1. 678 Oak St, Springfield, IL, 62712\n"
        "2. 789 Ash St, Springfield, IL, 62713\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID mno87654 for further communications.\n"
        "\n"
        "Example 5:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: PQR9012\n"
        "Assistant: Fetching the list of addresses associated with contract ID PQR9012...\n"
        "1. 890 Fir St, Springfield, IL, 62714\n"
        "2. 901 Spruce St, Springfield, IL, 62715\n"
        "User: Can you confirm the details of address 1?\n"
        "Assistant: The address is 890 Fir St, Springfield, IL, 62714. Please confirm if you would like to replace with this address.\n"
        "User: Yes, replace with address 1.\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID pqr90123 for further communications.\n"
        "\n"
        "Example 6:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: STU3456 and the new address is 345 Pine St\n"
        "Assistant: Please provide only your contract ID to proceed with replacing an existing address.\n"
        "User: STU3456\n"
        "Assistant: Fetching the list of addresses associated with contract ID STU3456...\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID stu34567 for further communications.\n"
        "\n"
        "Example 7:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: VWX6789\n"
        "Assistant: Fetching the list of addresses associated with contract ID VWX6789...\n"
        "1. 234 Birch St, Springfield, IL, 62718\n"
        "2. 345 Cedar St, Springfield, IL, 62719\n"
        "User: Nevermind, I don't want to change it.\n"
        "Assistant: No problem. If you need further assistance, please let me know.\n"
        "\n"
        "Example 8:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: YZA1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID YZA1234...\n"
        "1. 567 Elm St, Springfield, IL, 62720\n"
        "2. 678 Pine St, Springfield, IL, 62721\n"
        "User: Are you sure these are the correct addresses?\n"
        "Assistant: Yes, these are the addresses associated with contract ID YZA1234. Please select one address to replace.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID yza12345 for further communications.\n"
        "\n"
        "Example 9:\n"
        "User: I don't understand.\n"
        "Assistant: I'm here to assist you. I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 10:\n"
        "User: Please repeat.\n"
        "Assistant: Certainly! I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 11:\n"
        "User: Can you tell me what was done so far?\n"
        "Assistant: Sure, we have discussed the process of replacing an address associated with a contract ID. The provided contract ID was contract_id. We fetched the list of addresses associated with this contract ID, and you selected an address to replace. If you have further questions, please let me know.\n"
        "\n"
        "Example 12:\n"
        "User: Can we go back one step?\n"
        "Assistant: Sure, let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the address you would like to replace by entering the corresponding number.\n"
        "\n"
        "Example 13:\n"
        "User: I forgot my contract ID.\n"
        "Assistant: No problem. Please check your records and provide your contract ID so we can proceed with replacing the address.\n"
        "If you cannot find your contract ID, please contact support for further assistance.\n"
        "\n"
        "Example 14:\n"
        "User: How do you know it's not correct?\n"
        "Assistant: The provided contract ID did not match any records in our system. Please double-check your contract ID and provide it again.\n"
        "\n"
        "Example 15:\n"
        "User: Yes\n"
        "Assistant: Great! Please provide your contract ID so I can assist you with replacing an address from the existing list of addresses.\n"
        "\n"
        "Example 16:\n"
        "User: I selected the wrong number for the address.\n"
        "Assistant: No problem. Let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the correct address by entering the corresponding number.\n"
        "\n"
        "Example 17:\n"
        "User: 3\n"
        "Assistant: The number you selected is not valid. Please select a valid address number from the list:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Enter the number corresponding to the address you want to replace."
    ),
    ("user", "{user_input}"),
])


In [7]:
class AddressDetails(BaseModel):
    """Extracts Enitities such as Contract ID,full address , city , state, zipcode from the user input."""
    contract_id: str = Field(description="Contract ID")
    address_lane: str = Field(description="Address_lane provided")
    city: str = Field(description="name of the city ")
    state: str = Field(description="name of the State")
   # country: str = Field(description="name of the country")
    zipcode: int = Field(description="Zipcode or the location")


In [8]:
from langchain.memory import ConversationBufferMemory
model = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
tools = []
extraction_functions = [convert_pydantic_to_openai_function(AddressDetails)]
address_extraction_model = model.bind(functions=extraction_functions)
chain = replace_address_prompt | address_extraction_model | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=chain,tools = tools,verbose=True, memory=memory)
agent_executor.invoke({"user_input": ''})



> Entering new AgentExecutor chain...
I'm here to assist you. Please provide your contract ID so we can proceed with replacing an address from the existing list of addresses.

> Finished chain.


{'user_input': '',
 'chat_history': [HumanMessage(content=''),
  AIMessage(content="I'm here to assist you. Please provide your contract ID so we can proceed with replacing an address from the existing list of addresses.")],
 'output': "I'm here to assist you. Please provide your contract ID so we can proceed with replacing an address from the existing list of addresses."}

In [ ]:
# Run the agent executor with input
message = "I have slected to replace the address from an existing list, my contract ID is: "
print(message)
while True:
    user_input = input()
    if user_input in ['exit','bye']:
        break
    else:
        user_input = message +" "+ user_input
        response = agent_executor.invoke({
        "user_input": user_input
        })
        message = ""

print(response)

In [9]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, tool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

In [15]:
# Change Email Address Chain
email_prompt = ChatPromptTemplate.from_template(
    "Please provide your current email address and the new email address you would like to use."
)
change_email_chain = email_prompt | model | output_parser

class ChangeEmailInput(BaseModel):
    current_email: str = Field(description="Current email address")
    new_email: str = Field(description="New email address")

@tool("change_email_tool", args_schema=ChangeEmailInput)
def change_email(current_email: str, new_email: str) -> str:
    """Change the user's email address in the database."""
    # Logic to change email in the database
    return f"Email changed from {current_email} to {new_email}"

change_email_tool_instance = change_email

In [16]:
# Submit Inquiry Chain
inquiry_prompt = ChatPromptTemplate.from_template(
    "Please describe your inquiry in detail."
)
submit_inquiry_chain = inquiry_prompt | model | output_parser

class SubmitInquiryInput(BaseModel):
    inquiry: str = Field(description="User inquiry")

@tool("submit_inquiry_tool", args_schema=SubmitInquiryInput)
def submit_inquiry(inquiry: str) -> str:
    """Submit the user's inquiry."""
    # Logic to submit the inquiry
    return f"Inquiry submitted: {inquiry}"

submit_inquiry_tool_instance = submit_inquiry

In [17]:
general_prompt = ChatPromptTemplate.from_template("Sorry, I didn't understand your request. Please try again.")
general_chain = general_prompt | model | output_parser

In [35]:
replace_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with replacing an address from an existing list of addresses. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Fetch and present the list of addresses associated with the provided contract ID from the tenant table.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of replacing an address from an existing list of addresses.\n"
        "5. If the user asks about the conversation so far, answer within the context.\n"
        "6. If the user asks a question unrelated to address changes, politely ask the user to stay within the context of the conversation.\n"
        "7. If the user expresses confusion or asks for repetition, politely clarify and repeat the necessary information.\n"
        "8. If the user wants to go back one step, provide the previous stage's information.\n"
        "9. If the user selects an invalid option, politely ask them to select a valid option from the list.\n"
        "\n"
        "Example Conversations:\n"
        "\n"
        "Example 1:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: DEF4567\n"
        "Assistant: Fetching the list of addresses associated with contract ID DEF4567...\n"
        "1. 789 Pine St, Springfield, IL, 62706\n"
        "2. 101 Maple St, Springfield, IL, 62707\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID def56789 for further communications.\n"
        "\n"
        "Example 2:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: GHI8901\n"
        "Assistant: Fetching the list of addresses associated with contract ID GHI8901...\n"
        "1. 234 Elm St, Springfield, IL, 62708\n"
        "2. 345 Cedar St, Springfield, IL, 62709\n"
        "User: What time does the office close?\n"
        "Assistant: Please stay within the context of the conversation. Would you like to replace an address? Please select one address from the list.\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID ghi89101 for further communications.\n"
        "\n"
        "Example 3:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: JKL1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID JKL1234...\n"
        "1. 456 Birch St, Springfield, IL, 62710\n"
        "2. 567 Walnut St, Springfield, IL, 62711\n"
        "User: What did we talk about earlier?\n"
        "Assistant: We were discussing replacing your address. You provided contract ID JKL1234. Please select one address from the list.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID jkl12345 for further communications.\n"
        "\n"
        "Example 4:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: MNO5678\n"
        "Assistant: The provided contract ID does not match any records. Please check the contract ID and provide it again.\n"
        "User: MNO8765\n"
        "Assistant: Fetching the list of addresses associated with contract ID MNO8765...\n"
        "1. 678 Oak St, Springfield, IL, 62712\n"
        "2. 789 Ash St, Springfield, IL, 62713\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID mno87654 for further communications.\n"
        "\n"
        "Example 5:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: PQR9012\n"
        "Assistant: Fetching the list of addresses associated with contract ID PQR9012...\n"
        "1. 890 Fir St, Springfield, IL, 62714\n"
        "2. 901 Spruce St, Springfield, IL, 62715\n"
        "User: Can you confirm the details of address 1?\n"
        "Assistant: The address is 890 Fir St, Springfield, IL, 62714. Please confirm if you would like to replace with this address.\n"
        "User: Yes, replace with address 1.\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID pqr90123 for further communications.\n"
        "\n"
        "Example 6:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: STU3456 and the new address is 345 Pine St\n"
        "Assistant: Please provide only your contract ID to proceed with replacing an existing address.\n"
        "User: STU3456\n"
        "Assistant: Fetching the list of addresses associated with contract ID STU3456...\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "User: 2\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID stu34567 for further communications.\n"
        "\n"
        "Example 7:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: VWX6789\n"
        "Assistant: Fetching the list of addresses associated with contract ID VWX6789...\n"
        "1. 234 Birch St, Springfield, IL, 62718\n"
        "2. 345 Cedar St, Springfield, IL, 62719\n"
        "User: Nevermind, I don't want to change it.\n"
        "Assistant: No problem. If you need further assistance, please let me know.\n"
        "\n"
        "Example 8:\n"
        "User: I want to replace my address.\n"
        "Assistant: Sure, I can help with that. Please provide your contract ID.\n"
        "User: YZA1234\n"
        "Assistant: Fetching the list of addresses associated with contract ID YZA1234...\n"
        "1. 567 Elm St, Springfield, IL, 62720\n"
        "2. 678 Pine St, Springfield, IL, 62721\n"
        "User: Are you sure these are the correct addresses?\n"
        "Assistant: Yes, these are the addresses associated with contract ID YZA1234. Please select one address to replace.\n"
        "User: 1\n"
        "Assistant: Great, we have received your request to replace the address. Kindly note your reference ID yza12345 for further communications.\n"
        "\n"
        "Example 9:\n"
        "User: I don't understand.\n"
        "Assistant: I'm here to assist you. I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 10:\n"
        "User: Please repeat.\n"
        "Assistant: Certainly! I can help you replace an address from an existing list of addresses. Please provide your contract ID.\n"
        "\n"
        "Example 11:\n"
        "User: Can you tell me what was done so far?\n"
        "Assistant: Sure, we have discussed the process of replacing an address associated with a contract ID. The provided contract ID was contract_id. We fetched the list of addresses associated with this contract ID, and you selected an address to replace. If you have further questions, please let me know.\n"
        "\n"
        "Example 12:\n"
        "User: Can we go back one step?\n"
        "Assistant: Sure, let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the address you would like to replace by entering the corresponding number.\n"
        "\n"
        "Example 13:\n"
        "User: I forgot my contract ID.\n"
        "Assistant: No problem. Please check your records and provide your contract ID so we can proceed with replacing the address.\n"
        "If you cannot find your contract ID, please contact support for further assistance.\n"
        "\n"
        "Example 14:\n"
        "User: How do you know it's not correct?\n"
        "Assistant: The provided contract ID did not match any records in our system. Please double-check your contract ID and provide it again.\n"
        "\n"
        "Example 15:\n"
        "User: Yes\n"
        "Assistant: Great! Please provide your contract ID so I can assist you with replacing an address from the existing list of addresses.\n"
        "\n"
        "Example 16:\n"
        "User: I selected the wrong number for the address.\n"
        "Assistant: No problem. Let's go back to the previous step. Here are the addresses associated with contract ID contract_id:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Please select the correct address by entering the corresponding number.\n"
        "\n"
        "Example 17:\n"
        "User: 3\n"
        "Assistant: The number you selected is not valid. Please select a valid address number from the list:\n"
        "1. 123 Oak St, Springfield, IL, 62716\n"
        "2. 456 Maple St, Springfield, IL, 62717\n"
        "Enter the number corresponding to the address you want to replace."
    ),
    ("user", "{question}"),
])



class UpdateAddressInput(BaseModel):
    street: str = Field(description="Street address")
    city: str = Field(description="City")
    state: str = Field(description="State")
    zip_code: str = Field(description="Zip code")

@tool("update_address_tool", args_schema=UpdateAddressInput)
def update_address(street: str, city: str, state: str, zip_code: str) -> str:
    """Update the user's address in the database."""
    # Logic to update address in the database
    return f"Address updated to {street}, {city}, {state}, {zip_code}"

In [36]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
# Initialize the model and parser
tools = [update_address_tool_instance]
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
output_parser = OpenAIFunctionsAgentOutputParser()

update_address_chain = replace_address_prompt | model | output_parser
# result = update_address_chain.invoke({"question": "can you tell about yourself?"})
# print(result)

In [ ]:
update_address_chain.invoke{""}

In [37]:
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
agent_executor = AgentExecutor(agent=update_address_chain,tools = tools,verbose=True, memory=memory)
agent_executor.invoke({"question": 'what am i?'})



> Entering new AgentExecutor chain...
I'm here to assist you. How can I help you today?

> Finished chain.


{'question': 'what am i?',
 'chat_history': [HumanMessage(content='what am i?'),
  AIMessage(content="I'm here to assist you. How can I help you today?")],
 'output': "I'm here to assist you. How can I help you today?"}

In [38]:
agent_executor.invoke({"question": 'how can you help me?'})



> Entering new AgentExecutor chain...
I can assist you with replacing an address from an existing list of addresses. If you have a specific address you would like to update, please provide me with your contract ID, and I will fetch the list of addresses associated with that contract ID for you to choose from.

> Finished chain.


{'question': 'how can you help me?',
 'chat_history': [HumanMessage(content='what am i?'),
  AIMessage(content="I'm here to assist you. How can I help you today?"),
  HumanMessage(content='how can you help me?'),
  AIMessage(content='I can assist you with replacing an address from an existing list of addresses. If you have a specific address you would like to update, please provide me with your contract ID, and I will fetch the list of addresses associated with that contract ID for you to choose from.')],
 'output': 'I can assist you with replacing an address from an existing list of addresses. If you have a specific address you would like to update, please provide me with your contract ID, and I will fetch the list of addresses associated with that contract ID for you to choose from.'}

In [41]:
agent_executor.invoke({"question": 'okay'})



> Entering new AgentExecutor chain...
If you have any further questions or need assistance with replacing an address, please feel free to let me know.

> Finished chain.


{'question': 'okay',
 'chat_history': [HumanMessage(content='what am i?'),
  AIMessage(content="I'm here to assist you. How can I help you today?"),
  HumanMessage(content='how can you help me?'),
  AIMessage(content='I can assist you with replacing an address from an existing list of addresses. If you have a specific address you would like to update, please provide me with your contract ID, and I will fetch the list of addresses associated with that contract ID for you to choose from.'),
  HumanMessage(content='the contract id is CAT1223'),
  AIMessage(content='Fetching the list of addresses associated with contract ID CAT1223...\n1. 789 Pine St, Springfield, IL, 62706\n2. 101 Maple St, Springfield, IL, 62707\n\nPlease select the address you would like to replace by entering the corresponding number.'),
  HumanMessage(content='2'),
  AIMessage(content='Great, we have received your request to replace the address associated with contract ID CAT1223. Kindly note your reference ID for fur

In [70]:
classification_prompt = ChatPromptTemplate.from_messages(
    [("system","classify the user input into either of the three classes only \n 1)inquiry\n 2)change email\n 3) update address \n"
               "If neither of the above three classes then classify it as others\n"
                "The output should adhere to the labels as mentioned above only\n"
                 "If you dont clearly understand, prompt the user to repeat again"),
     ("human","{question}")
    ]            
              
)

In [71]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

In [72]:
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

topic_chain = classification_prompt|model|OpenAIFunctionsAgentOutputParser()
topic_chain.invoke({'question':"i need to change my address"})

AgentFinish(return_values={'output': 'update address'}, log='update address')

In [59]:
#topic_chain.invoke({'question':"I need to change my address"})
topic_classifier_chain = RunnablePassthrough.assign(input=topic_chain)
result = topic_classifier_chain.invoke({'question':"i need to change my address"})
result

{'question': 'i need to change my address',
 'input': AgentFinish(return_values={'output': 'change address'}, log='change address')}

In [65]:
result['input'].return_values['output']

'change address'

In [73]:
# Create the routing branch
branch = RunnableBranch(
    (lambda x: "update address" in result['input'].return_values['output'].lower(), agent_executor),
    (lambda x: "change email" in result['input'].return_values['output'].lower(), change_email_chain | change_email_tool_instance),
    (lambda x: "inquiry" in result['input'].return_values['output'].lower(), submit_inquiry_chain | submit_inquiry_tool_instance),
    general_chain
)

# Full chain with topic classifier
full_chain = topic_classifier_chain|branch

In [75]:
# Example usage
user_input = "i need to update my address??"
response = full_chain.invoke({"question": user_input})
print(response)

No problem! How can I assist you today?


In [ ]:
import sqlite3
from langchain_community.utilities import SQLDatabase
# from langchain_experimental.sql import SQLDatabaseChain

In [8]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('landlords.db')
cursor = conn.cursor()

# Create the table
cursor.execute('''
CREATE TABLE IF NOT EXISTS address (
    Contract_ID INTEGER,
    Date_of_Updation TEXT,
    Address TEXT
)
''')

# Insert dummy data
dummy_data = [
    (1, '2024-01-01', '123 Main St, Springfield'),
    (1, '2024-02-01', '124 Main St, Springfield'),
    (2, '2024-01-15', '456 Elm St, Shelbyville'),
    (2, '2024-03-20', '457 Elm St, Shelbyville'),
    (3, '2024-02-10', '789 Maple St, Capital City')
]

cursor.executemany('''
INSERT INTO address (Contract_ID, Date_of_Updation, Address)
VALUES (?, ?, ?)
''', dummy_data)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [9]:
# Function to query the table
def query_addresses(contract_id=None):
    # Connect to the SQLite database
    conn = sqlite3.connect('landlords.db')
    cursor = conn.cursor()

    if contract_id:
        # Query for a specific Contract_ID
        cursor.execute('SELECT * FROM address WHERE Contract_ID = ?', (contract_id,))
    else:
        # Query all records
        cursor.execute('SELECT * FROM address')
    
    # Fetch all results
    results = cursor.fetchall()
    
    # Close the connection
    conn.close()
    
    return results

In [10]:

# Example usage
print("All records:")
for row in query_addresses():
    print(row)

All records:
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')
(2, '2024-01-15', '456 Elm St, Shelbyville')
(2, '2024-03-20', '457 Elm St, Shelbyville')
(3, '2024-02-10', '789 Maple St, Capital City')
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')
(2, '2024-01-15', '456 Elm St, Shelbyville')
(2, '2024-03-20', '457 Elm St, Shelbyville')
(3, '2024-02-10', '789 Maple St, Capital City')


In [11]:


# Example usage
print("All records:")
for row in query_addresses():
    print(row)

print("\nRecords for Contract_ID 1:")
for row in query_addresses(1):
    print(row)

All records:
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')
(2, '2024-01-15', '456 Elm St, Shelbyville')
(2, '2024-03-20', '457 Elm St, Shelbyville')
(3, '2024-02-10', '789 Maple St, Capital City')
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')
(2, '2024-01-15', '456 Elm St, Shelbyville')
(2, '2024-03-20', '457 Elm St, Shelbyville')
(3, '2024-02-10', '789 Maple St, Capital City')

Records for Contract_ID 1:
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')
(1, '2024-01-01', '123 Main St, Springfield')
(1, '2024-02-01', '124 Main St, Springfield')


In [12]:
db = SQLDatabase.from_uri("sqlite:///landlords.db")
print(db.dialect)
db.run('SELECT * FROM address LIMIT 3;')

sqlite


"[(1, '2024-01-01', '123 Main St, Springfield'), (1, '2024-02-01', '124 Main St, Springfield'), (2, '2024-01-15', '456 Elm St, Shelbyville')]"

In [13]:
print(db.get_usable_table_names())
print(db.get_table_info())

['address']

CREATE TABLE address (
	"Contract_ID" INTEGER, 
	"Date_of_Updation" TEXT, 
	"Address" TEXT
)

/*
3 rows from address table:
Contract_ID	Date_of_Updation	Address
1	2024-01-01	123 Main St, Springfield
1	2024-02-01	124 Main St, Springfield
2	2024-01-15	456 Elm St, Shelbyville
*/


In [14]:
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = "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"

In [15]:
from langchain.chains import create_sql_query_chain
generate_query = create_sql_query_chain(model,db)

query = generate_query.invoke({"question": "retrieve the addresses for contract id 1"})
print(query)

SELECT "Address" 
FROM address 
WHERE "Contract_ID" = 1
LIMIT 5;


In [16]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
print(execute_query.invoke(query))

[('123 Main St, Springfield',), ('124 Main St, Springfield',), ('123 Main St, Springfield',), ('124 Main St, Springfield',)]


In [17]:
database_chain = generate_query|execute_query
database_chain.invoke({"question": "retrieve the addresses for contract id 1"})

"[('123 Main St, Springfield',), ('124 Main St, Springfield',), ('123 Main St, Springfield',), ('124 Main St, Springfield',)]"

In [18]:
database_chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [42]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [20]:
answer_prompt = PromptTemplate.from_template(
                """Given the user question, corresponding SQL query, and SQL result. Answer the user question in a funny way. 
                
                Question:{question}
                SQL Query:{query}
                SQL Result{result}
                Answer:
                """)

rephrase_answer = answer_prompt|model|StrOutputParser()

chain = (
        RunnablePassthrough.assign(query = generate_query).assign(
        result = itemgetter("query")|execute_query)
        |rephrase_answer
)

chain.invoke({"question":"How many unique contract ID do i have?"})

"Looks like you have 3 unique contract IDs, but remember, it's not about the quantity of contracts, it's about the quality of the deals you make! Quality over quantity, my friend."

#### Adding a few shot examples 

In [22]:
examples = [
        {
            "input":"List all the addresses",
            "query":"SELECT Address from address"
        },
        
        {
            "input":"Show me the unique contract Id's",
            "query":"SELECT DISTINCT(Contract_ID)from address"
        },
        
        {
            "input": "get me the result associated with the contract ID 1",
            "query": "SELECT Address from address WHERE Contract_ID = 1"
        },
    
        {
            "input": "which are the date when the address added for contact id 1",
            "query":"SELCT Date_of_Updation from address WHERE Contract_ID = 1"
        }
        
]

In [38]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder , FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
                [
                    ("human","{input}\nSQLQuery"),
                    ("ai","{query}")
                ])

few_shot_prompt = FewShotChatMessagePromptTemplate(
                example_prompt = example_prompt,
                examples = examples,
                input_variables=['input','top_k'])

print(few_shot_prompt.format(input = "when was the address added for the contact id 1"))

Human: List all the addresses
SQLQuery
AI: SELECT Address from address
Human: Show me the unique contract Id's
SQLQuery
AI: SELECT DISTINCT(Contract_ID)from address
Human: get me the result associated with the contract ID 1
SQLQuery
AI: SELECT Address from address WHERE Contract_ID = 1
Human: which are the date when the address added for contact id 1
SQLQuery
AI: SELCT Date_of_Updation from address WHERE Contract_ID = 1


#### Dynamic Few shot prompt Selection

In [39]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

In [ ]:
vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=vectorstore,
    input_keys=["input"], 
    k=3,  # Number of examples to select
)
example_selector.select_examples({"input": "When was the last address updated for contract id 1?"})

In [36]:
final_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system","You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\n"
                               "Here is the relevant table info:\n{table_info}"),
                    few_shot_prompt,
                    ('human',"{input}")
                    
                ])

In [37]:
print(final_prompt.format(input="Get me the latest dated for contract id 1", table_info = "Dummy Table",top_k=3))

System: You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Here is the relevant table info:
Dummy Table
Human: List all the addresses
SQLQuery
AI: SELECT Address from address
Human: Show me the unique contract Id's
SQLQuery
AI: SELECT DISTINCT(Contract_ID)from address
Human: get me the result associated with the contract ID 1
SQLQuery
AI: SELECT Address from address WHERE Contract_ID = 1
Human: which are the date when the address added for contact id 1
SQLQuery
AI: SELCT Date_of_Updation from address WHERE Contract_ID = 1
Human: Get me the latest dated for contract id 1


In [46]:
generate_query = create_sql_query_chain(model,db,final_prompt)
chain = (
        RunnablePassthrough.assign(query = generate_query).assign(
        result=itemgetter("query")|execute_query)
       | rephrase_answer)

chain.invoke({"question":"when was the last update of address made for contract id 1",
              "table_info" : "Dummy Table",
              "top_k": '3'})

"Oh, the last time someone decided to change their address for contract id 1 was in the distant future of 2024. Looks like they're really planning ahead!"

### Email

In [12]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import logging
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import BaseTool

In [16]:
from pydantic import BaseModel, Field, EmailStr
from typing import List,Optional
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [17]:
from pydantic import BaseModel, Field, EmailStr
from typing import Optional
from email.message import EmailMessage
import ssl
import smtplib
import logging
from langchain.tools import BaseTool

# Define the EmailAttributes model
class EmailAttributes(BaseModel):
    sender_email: EmailStr = Field(description="The sender's email address")
    receiver_email: EmailStr = Field(description="The receiver's email address")
#     subject: Optional[str] = Field(default="No Subject", description="The subject of the email")
#     body: Optional[str] = Field(default="No Body Content", description="The body content of the email")

# Define the EmailSenderTool class inheriting from BaseTool
class EmailSenderTool(BaseTool):
    name = "email_sender"
    description = "A tool to send emails with reference ID for address updates."

    smtp_server: str = "smtp.gmail.com"
    smtp_port: int = 465
    smtp_username: str = "chatbot.testing.05@gmail.com"
    smtp_password: str = "dlwg ovip zkso dyow" #app-specific password

    def __init__(self, smtp_server: str = None, smtp_port: int = None, smtp_username: str = None, smtp_password: str = None):
        super().__init__()
        self.smtp_server = smtp_server or self.smtp_server
        self.smtp_port = smtp_port or self.smtp_port
        self.smtp_username = smtp_username or self.smtp_username
        self.smtp_password = smtp_password or self.smtp_password
        # Configure logging
        logging.basicConfig(level=logging.INFO)

    def send_email(self, email_attributes: EmailAttributes, reference_id: str) -> str:
        subject = 'LLP- address Update'
        body = f"""\
        We have received your request for updating the address for your contract id, kindly find your reference id for further communications.

        Reference ID: {reference_id}

        Regards,
        Landlord portal
        """
        
        instance = EmailMessage()
        instance["From"] = self.smtp_username
        instance["To"] = email_attributes.receiver_email
        instance["Subject"] = subject
        instance.set_content(body)
        
        context = ssl.create_default_context()

        try:
            with smtplib.SMTP_SSL(self.smtp_server, self.smtp_port, context=context) as smtp:
                smtp.login(self.smtp_username, self.smtp_password)
                smtp.sendmail(self.smtp_username, email_attributes.receiver_email, instance.as_string())
                logging.info(f"Email sent successfully to {email_attributes.receiver_email} with reference ID {reference_id}.")
                return "Email sent successfully."
        except Exception as e:
            logging.error(f"Failed to send email to {email_attributes.receiver_email} with reference ID {reference_id}. Error: {e}")
            return f"Failed to send email: {e}"

    def _run(self, email_attributes: EmailAttributes, reference_id: str) -> str:
        return self.send_email(email_attributes, reference_id)

    async def _arun(self, email_attributes: EmailAttributes, reference_id: str) -> str:
        return self.send_email(email_attributes, reference_id)

In [ ]:
# Create an instance of the EmailSenderTool
email_sender_tool = EmailSenderTool()

In [15]:
# Define a method to use this tool in Langchain format
def use_email_sender_tool(email_attributes: EmailAttributes, reference_id: str) -> str:
    return email_sender_tool._run(email_attributes, reference_id)

In [16]:
# Example usage
email_attributes = EmailAttributes(
    sender_email="chatbot.testing.05@gmail.com",
    receiver_email="freelancing.aiml@gmail.com"
)
reference_id = "123456"

# Send email using the tool
result = use_email_sender_tool(email_attributes, reference_id)
print(result)

INFO:root:Email sent successfully to freelancing.aiml@gmail.com with reference ID 123456.


Email sent successfully.


In [ ]:
# Define the combined chat prompt template for adding a new address
add_address_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("system",
        "You are a helpful and polite landlord portal support agent. Your task is to assist users with updating their address information. Follow these guidelines strictly:\n"
        "1. Gather the contract ID information from the user.\n"
        "2. Gather address information: address lane, city, state, and zip code.\n"
        "3. Do not ask for any other information.\n"
        "4. Do not answer any questions outside of address changes.\n"
        "5. If the user asks a question unrelated to address changes, respond with: 'My functionality is limited to address changes.'\n"
        "\n"
        "If the user wants to add a new address, follow these steps:\n"
        "Step 1: Ask for the contract ID if it has not already been provided.\n"
        "Step 2: Ask for the full address including address lane, city, state (check if the state is located in the USA), and zip code.\n"
        "Step 3: Validate that all fields are provided. If any field is missing, prompt the user to provide the missing fields one by one.\n"
        "Step 4: Generate a reference ID and respond with: 'Your request for adding a new address is received. Your reference ID is [reference_id] for future communications. Thank you!'\n"
        "\n"
        "Example Conversation:\n"
        "Assistant: Can you provide me with your Contract ID?\n"
        "Human: ABC10023\n"
        "Assistant: Do you want to replace an existing address or add a new address?\n"
        "Human: I want to add an address.\n"
        "Assistant: You chose to add a new address. Please provide the full address including address lane, city, state, and zip code.\n"
        "User: 123 Main St, Springfield\n"
        "Assistant: It seems you missed the state and zip code. Please provide the state.\n"
        "Human: Illinois\n"
        "Assistant: Please provide the zip code.\n"
        "Human: 62704\n"
        "Assistant: Thank you. Your request for adding a new address is received. Your reference ID is [reference_id] for future communications. Thank you!\n"
    ),
    ("user", "{user_input}"),
])